# 📑 Tutorials for PyPOTS Classification Models

## 📀 Preparing the **PhysioNet-2012** dataset for this tutorial

In [1]:
from pypots.data.generating import gene_physionet2012
from pypots.utils.random import set_random_seed
from global_config import RANDOM_SEED

set_random_seed(RANDOM_SEED)

# Load the PhysioNet-2012 dataset, disable artificially-missing values for evaluation
physionet2012_dataset = gene_physionet2012(artificially_missing_rate=0.1)
 
# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you, 
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

/opt/homebrew/anaconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-16 23:57:08 [INFO]: Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Database)...
2023-05-16 23:57:08 [INFO]: Starting preprocessing physionet_2012...


Dataset physionet_2012 has already been downloaded. Processing directly...
Dataset physionet_2012 has already been cached. Loading from cache directly...
Loaded successfully!
dict_keys(['n_classes', 'n_steps', 'n_features', 'train_X', 'train_y', 'val_X', 'val_y', 'test_X', 'test_y', 'scaler', 'test_X_intact', 'test_X_indicating_mask', 'val_X_intact', 'val_X_indicating_mask'])


In [2]:
# Assemble the datasets for training, validating, and testing.

dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
    "y": physionet2012_dataset['train_y'],
}

dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "y": physionet2012_dataset['val_y'],
}

dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "y": physionet2012_dataset['test_y'],
}

## 🚀 An exmaple of **Raindrop** for classification

In [3]:
from pypots.optim import Adam
from pypots.classification import Raindrop

# initialize the model
raindrop = Raindrop(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_classes=physionet2012_dataset["n_classes"],
    n_layers=2,
    d_model=physionet2012_dataset["n_features"] * 4,
    d_inner=256,
    n_heads=2,
    dropout=0.3,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default, PyPOTS will automatically assign the best device for you.
    # Set it to 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices.
    device='cpu',  
    # set the path for saving tensorboard and trained model files 
    saving_path="tutorial_results/classification/raindrop",
    model_saving_strategy="best", # only save the best model after training finished. 
                                  # You can also set it as "better" to save models performing better ever during training.
)

2023-05-16 23:57:19 [INFO]: the trained model will be saved to tutorial_results/classification/raindrop/20230516_T235719
2023-05-16 23:57:19 [INFO]: the tensorboard file will be saved to tutorial_results/classification/raindrop/20230516_T235719/tensorboard
2023-05-16 23:57:19 [INFO]: Model initialized successfully with the number of trainable parameters: 1,415,006


In [4]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
raindrop.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2023-05-16 23:58:09 [INFO]: epoch 0: training loss 0.4002, validating loss 0.3761
2023-05-16 23:58:56 [INFO]: epoch 1: training loss 0.3380, validating loss 0.3567
2023-05-16 23:59:42 [INFO]: epoch 2: training loss 0.3210, validating loss 0.3763
2023-05-17 00:00:30 [INFO]: epoch 3: training loss 0.3118, validating loss 0.3575
2023-05-17 00:01:17 [INFO]: epoch 4: training loss 0.3081, validating loss 0.3604
2023-05-17 00:01:17 [INFO]: Exceeded the training patience. Terminating the training procedure...
2023-05-17 00:01:17 [INFO]: Finished training.
2023-05-17 00:01:17 [INFO]: Saved the model to tutorial_results/classification/raindrop/20230516_T235719/Raindrop.pypots.


In [5]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
raindrop_prediction = raindrop.classify(dataset_for_testing)

In [6]:
from pypots.utils.metrics import cal_binary_classification_metrics

# calculate mean absolute error on the ground truth (artificially-missing values)
metrics = cal_binary_classification_metrics(raindrop_prediction, dataset_for_testing["y"])
print("Testing classification metrics: \n"
    f'ROC_AUC: {metrics["roc_auc"]}, \n'
    f'PR_AUC: {metrics["pr_auc"]},\n'
    f'F1: {metrics["f1"]},\n'
    f'Precision: {metrics["precision"]},\n'
    f'Recall: {metrics["recall"]},\n'
)

Testing classification metrics: 
ROC_AUC: 0.8254049013391027, 
PR_AUC: 0.45063928834235356,
F1: 0.2766990291262136,
Precision: 0.6404494382022472,
Recall: 0.17647058823529413,


## 🚀 An exmaple of **BRITS** for classification

In [7]:
from pypots.optim import Adam
from pypots.classification import BRITS

# initialize the model
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'], 
    n_features=physionet2012_dataset['n_features'], 
    n_classes=physionet2012_dataset["n_classes"],
    rnn_hidden_size=256,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default, PyPOTS will automatically assign the best device for you.
    # Set it to 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices.
    device='cpu',  
    # set the path for saving tensorboard and trained model files 
    saving_path="tutorial_results/classification/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2023-05-17 00:01:24 [INFO]: the trained model will be saved to tutorial_results/classification/brits/20230517_T000124
2023-05-17 00:01:24 [INFO]: the tensorboard file will be saved to tutorial_results/classification/brits/20230517_T000124/tensorboard
2023-05-17 00:01:24 [INFO]: Model initialized successfully with the number of trainable parameters: 730,612


In [8]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2023-05-17 00:02:18 [INFO]: epoch 0: training loss 0.9090, validating loss 0.8378
2023-05-17 00:03:05 [INFO]: epoch 1: training loss 0.7656, validating loss 0.7810
2023-05-17 00:03:53 [INFO]: epoch 2: training loss 0.7171, validating loss 0.7480
2023-05-17 00:04:40 [INFO]: epoch 3: training loss 0.6937, validating loss 0.7508
2023-05-17 00:05:26 [INFO]: epoch 4: training loss 0.6767, validating loss 0.7394
2023-05-17 00:06:14 [INFO]: epoch 5: training loss 0.6641, validating loss 0.7442
2023-05-17 00:07:01 [INFO]: epoch 6: training loss 0.6520, validating loss 0.7427
2023-05-17 00:07:48 [INFO]: epoch 7: training loss 0.6334, validating loss 0.7613
2023-05-17 00:07:48 [INFO]: Exceeded the training patience. Terminating the training procedure...
2023-05-17 00:07:48 [INFO]: Finished training.
2023-05-17 00:07:48 [INFO]: Saved the model to tutorial_results/classification/brits/20230517_T000124/BRITS.pypots.


In [9]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
brits_prediction = brits.classify(dataset_for_testing)

In [10]:
from pypots.utils.metrics import cal_binary_classification_metrics

# calculate mean absolute error on the ground truth (artificially-missing values)

metrics = cal_binary_classification_metrics(brits_prediction, dataset_for_testing["y"])
print("Testing classification metrics: \n"
    f'ROC_AUC: {metrics["roc_auc"]}, \n'
    f'PR_AUC: {metrics["pr_auc"]},\n'
    f'F1: {metrics["f1"]},\n'
    f'Precision: {metrics["precision"]},\n'
    f'Recall: {metrics["recall"]},\n'
)

Testing classification metrics: 
ROC_AUC: 0.8314685366854413, 
PR_AUC: 0.45516800127428686,
F1: 0.1904761904761905,
Precision: 0.6545454545454545,
Recall: 0.11145510835913312,


## 🚀 An exmaple of **GRUD** for classification

In [11]:
from pypots.optim import Adam
from pypots.classification import GRUD

# initialize the model
grud = GRUD(
    n_steps=physionet2012_dataset['n_steps'], 
    n_features=physionet2012_dataset['n_features'], 
    n_classes=physionet2012_dataset["n_classes"],
    rnn_hidden_size=32, 
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default, PyPOTS will automatically assign the best device for you.
    # Set it to 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices.
    device='cpu',  
    # set the path for saving tensorboard and trained model files 
    saving_path="tutorial_results/classification/grud",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2023-05-17 00:07:55 [INFO]: the trained model will be saved to tutorial_results/classification/grud/20230517_T000755
2023-05-17 00:07:55 [INFO]: the tensorboard file will be saved to tutorial_results/classification/grud/20230517_T000755/tensorboard
2023-05-17 00:07:55 [INFO]: Model initialized successfully with the number of trainable parameters: 16,128


In [12]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
grud.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2023-05-17 00:08:03 [INFO]: epoch 0: training loss 0.3933, validating loss 0.4098
2023-05-17 00:08:06 [INFO]: epoch 1: training loss 0.3361, validating loss 0.3488
2023-05-17 00:08:10 [INFO]: epoch 2: training loss 0.3127, validating loss 0.3942
2023-05-17 00:08:14 [INFO]: epoch 3: training loss 0.2996, validating loss 0.3758
2023-05-17 00:08:18 [INFO]: epoch 4: training loss 0.2923, validating loss 0.4225
2023-05-17 00:08:18 [INFO]: Exceeded the training patience. Terminating the training procedure...
2023-05-17 00:08:18 [INFO]: Finished training.
2023-05-17 00:08:18 [INFO]: Saved the model to tutorial_results/classification/grud/20230517_T000755/GRUD.pypots.


In [13]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
grud_prediction = grud.classify(dataset_for_testing)

In [14]:
from pypots.utils.metrics import cal_binary_classification_metrics

# calculate mean absolute error on the ground truth (artificially-missing values)

metrics = cal_binary_classification_metrics(grud_prediction, dataset_for_testing["y"])
print("Testing classification metrics: \n"
    f'ROC_AUC: {metrics["roc_auc"]}, \n'
    f'PR_AUC: {metrics["pr_auc"]},\n'
    f'F1: {metrics["f1"]},\n'
    f'Precision: {metrics["precision"]},\n'
    f'Recall: {metrics["recall"]},\n'
)

Testing classification metrics: 
ROC_AUC: 0.7703040023872582, 
PR_AUC: 0.42444960120448355,
F1: 0.3355408388520972,
Precision: 0.5846153846153846,
Recall: 0.23529411764705882,
